### Setup ###

In [1]:
# Markdown: puller_signup.ipynb
import mysql.connector
import pandas as pd
import bcrypt
from datetime import datetime, timedelta

DB_CONFIG = {"host":"localhost","user":"root","password":"40418","database":"aeras"}
def get_conn(): return mysql.connector.connect(**DB_CONFIG)
def now_bd(): return datetime.utcnow() + timedelta(hours=6)

# Puller ID generator (your chosen PLR format)
def gen_pullerid_from(signup_date_str, phone, sequence):
    # signup_date_str expected "YYYY-MM-DD"
    d = datetime.strptime(signup_date_str, "%Y-%m-%d").strftime("%Y%m%d")
    return f"PLR{d}-{str(sequence).zfill(4)}"

# Password hashing (bcrypt)
def hash_password(plain_password):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(plain_password.encode('utf-8'), salt).decode('utf-8')

def verify_password(plain_password, hashed):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed.encode('utf-8'))

# Example: add a puller
def add_puller(name, vehicle, phone, plain_password, signup_date_str, sequence):
    pullerid = gen_pullerid_from(signup_date_str, phone, sequence)
    phash = hash_password(plain_password)
    conn = get_conn()
    now = now_bd()
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO pullers (pullerid, name, vehicle, password_hash, points, sign_up_date, phone, ride_count, created_at)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (pullerid, name, vehicle, phash, 0, signup_date_str, phone, 0, now))
        conn.commit()
    conn.close()
    return pullerid

# Example usage
# new_id = add_puller("Kabul Karim", "CHA-C-11-5468", "01313731509", "12345", "2025-11-13", 1)
# print("Added puller:", new_id)


In [2]:
# Show pullers table
conn = get_conn()
df = pd.read_sql("SELECT id, pullerid, name, vehicle, points, ride_count, sign_up_date FROM pullers;", conn)
conn.close()
print(df.head())


C:\Users\dibya\AppData\Local\Temp\ipykernel_11396\4235068786.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT id, pullerid, name, vehicle, points, ride_count, sign_up_date FROM pullers;", conn)


DatabaseError: Execution failed on sql 'SELECT id, pullerid, name, vehicle, points, ride_count, sign_up_date FROM pullers;': 1054 (42S22): Unknown column 'id' in 'field list'